In [ ]:
import SQAOA
import helpers

import numpy as np
import networkx as nx
import pickle
import time
import itertools
import os

In [ ]:
np.set_printoptions(legacy="1.25")

os.makedirs("datasets", exist_ok=True)
os.makedirs("results", exist_ok=True)
os.makedirs("init_points", exist_ok=True)

In [ ]:
t_list = [0.1]  # Threshold for nucleus sampling
p_list = [1]  # Depth of SQAOA
N_list = [3]  # Number of nodes

methods = ["Powell"]  # Optimizer
same_parameters_list = [False]  # Use same parameters in all sub-problems, set to False in paper
use_alpha_list = [False]  # Parameterize transition operator, set to False in paper
use_const_gamma_list = [True]  # Assign weights and parameters to individual nodes for phase-separation, set to True in paper

graph_type = "ER"  # Type of graphs ("ER" or "complete").
num_graphs = 50  # Number of graphs in dataset
budget = 500  # Budget for optimizer
restarts = 5  # Number of times the optimization procedure is conducted before returning the best result
simulation = "sampling"  # How the quantum circuit is simulated. Set to "sampling" in paper, simulating the circuit by repeatedly sampling from its probability distribution (other option is "statevector")
shots = 1000  # When using "sampling" for the simulation, number of shots used to estimate the probability distribution
looping = False  # Solve problem with number of clusters restricted to 1,...,n and takes best result, set to False in paper
init_point = "optimal"  # Initialization of parameters. Set to "optimal" in paper, taking parameters for the instance with all negative costs as initial point (other option is "random")
use_bounds = False  # Restrict parameters to [0,2pi], set to False in paper

experiments = [
    element
    for element in itertools.product(
        t_list,
        p_list,
        N_list,
        methods,
        same_parameters_list,
        use_alpha_list,
        use_const_gamma_list,
    )
]

In [ ]:
for N in N_list:
    if graph_type == "ER":
        helpers.data_set_generation_ER(N, 0.5, num_graphs)
    elif graph_type == "complete":
        helpers.data_set_generation(N, num_graphs)
    else:
        raise Exception(f"graph_type must be 'ER' or 'complete', not {graph_type}.")

In [ ]:
final_results = {}
summary_results = {}

for experiment in experiments:
    t = experiment[0]
    p = experiment[1]
    N = experiment[2]
    method = experiment[3]
    same_parameters = experiment[4]
    use_alpha = experiment[5]
    use_const_gamma = experiment[6]

    final_results[experiment] = {
        "max_agree": [],
        "function_evaluations": [],
        "optimal_value": [],
        "optimal_solution": [],
        "approximation_ratio": [],
        "time": [],
    }

    if graph_type == "ER":
        data_load = pickle.load(open(f"datasets/ER_{N}_{num_graphs}_50.p", "rb"))
    elif graph_type == "complete":
        data_load = pickle.load(open(f"datasets/complete_{N}_{num_graphs}.p", "rb"))
    else:
        raise Exception(f"graph_type must be 'ER' or 'complete', not {graph_type}.")

    max_agrees = np.zeros(num_graphs)
    current_init_point = None

    for graph_number in range(num_graphs):
        G = nx.from_numpy_array(data_load[graph_number])

        objective = SQAOA.BlackBoxObjective(
            G,
            len(G.nodes),
            p,
            t,
            same_parameters=same_parameters,
            use_alpha=use_alpha,
            use_const_gamma=use_const_gamma,
            shots=shots,
            simulation=simulation,
        )

        if current_init_point is None:
            if init_point == "optimal":
                current_init_point = objective.optimal_init_point(
                    G, method, budget, restarts, looping=looping, use_bounds=use_bounds
                )
            else:
                current_init_point = init_point

        start = time.time()
        max_agree, optpar, nfev = helpers.maximize_agree(
            objective,
            G,
            method=method,
            budget=budget,
            looping=looping,
            restarts=restarts,
            init_point=current_init_point,
            use_bounds=use_bounds,
        )
        end = time.time()

        optimal_value, optimal_solution = helpers.max_agree(G)

        print(
            f"{N};{graph_number};{p};{nfev};{max_agree};{optimal_value};{optpar};{end - start};{objective.simulation_time}"
        )

        final_results[experiment]["max_agree"].append(max_agree)
        final_results[experiment]["function_evaluations"].append(nfev)
        final_results[experiment]["optimal_value"].append(optimal_value)
        final_results[experiment]["optimal_solution"].append(optimal_solution)
        final_results[experiment]["approximation_ratio"].append(
            max_agree / optimal_value
        )
        final_results[experiment]["time"].append(end - start)

    summary_result = [
        np.mean(final_results[experiment]["approximation_ratio"]),
        np.std(final_results[experiment]["approximation_ratio"]),
        np.mean(final_results[experiment]["time"]),
        np.std(final_results[experiment]["time"]),
    ]

    print(summary_result)
    summary_results[experiment] = summary_result

    if graph_type == "ER":
        file = f"results/ER_SQAOA_{experiment}.txt"
    else:
        data_load = file = f"results/complete_SQAOA_{experiment}.txt"

    with open(file, "a+") as fp:
        fp.writelines(
            [f"{final_results[experiment]}", "\n", f"{summary_results[experiment]}"]
        )